In [1]:
#!pip install pyproj
#!pip install request

# MANUAL SKETCHBOOK

La finalidad de este software es completar la mayor cantidad de información del estandar Darwin Core a partir de un par de coordenadas, *importante hacer notar que este software hasta el momento no realiza ningun tipo de georreferenciación, solo rellena datos*, esto se realiza a partir de un set de coordenadas, esta pueden ser del campo verbatimCoordinates o decimalLatitude y decimalLongitude, pertenecientes al estandar. Con estos datos será completar los siguientes campos, si es que se encuentra la información necesaria. 

- higherGeography 
- continent 
- country 
- countryCode 
- stateProvince 
- county 
- municipality 
- decimalLongitude, decimalLatitude
- geodeticDatum 
- coordinatePrecision 
- verbatimLatitude, verbatimLongitude
- verbatimCoordinateSystem 

Hasta el momento no se ha podido probar la localización de localidades marinas, pero es un trabajo en progreso. 

Este script se divide en dos clases

- Preparación de la base de datos (db_prep)
- Transformación de coordenadas (coord_transform) 

## Preparación de la base de datos

Esta clase recibe el nombre de "db_prep" y necesita como entrada una base de datos con algunos componentes mínimos del estandar Darwin Core (DwC) para su realización.



## Transformación de coordenadas 

- El campo verbatimCoordintes debe tener las coordenadas en el siguiente formato (dejar en georeferenceRemarks las coordenadas como vienen en su etiqueta original)
        -En el caso de coordenadas en grados minutos y segundos gg°mm'ss" S, gg°mm'ss" W o gg°mm'ss" N, gg°mm'ss" E la importancia radica que debes estar separadas por una coma, misma situación para grados y minutos decimales
        -En el caso de UTM deben ser escritas de la siguiente forma 308617 6272156 19S, la importancia esta que se encuentren separadas por espacio. 
    

In [2]:
from GeoParser import *
import pandas as pd
pd.set_option("display.max_rows", None, "display.max_columns", None)

In [3]:
data=pd.read_csv("GeoRefTest.csv")
data

,higherGeography,dynamicProperties,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,island,locality,verbatimLocality,minimumElevationInMeters,maximumElevationInMeters,verbatimElevation,minimumDepthInMeters,maximumDepthInMeters,verbatimDepth,locationRemarks,decimalLatitude,decimalLongitude,geodeticDatum,coordinateUncertaintyInMeters,coordinatePrecision,footprintSRS,footprintWKT,verbatimCoordinates,verbatimLatitude,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid
0,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Lagunita en Ruta 181, 6 km al Este de Lonquimay",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.453318,-71.292183,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TxmVXS0HuObUcicDzF9JBaWNHK4C/tiPdf9jxpTLMqU=
1,NaN,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Reserva Nacional Los Huemules de Niblinto, Coi...",R.F. Niblinto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-36.728678,-71.496973,(WGS84) World Geodetic System 1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,WRYd0YGi5ik2bvvJRhC1Ynf4OyfzirN66qbhuD3Fgfc=
2,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,150.0,156.0,NaN,NaN,-38.846400,-73.788900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TtMBF2hQMfzrgMVqyIbTHRP0UF9v4Qn8uNonMtuye/s=
3,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,464.0,500.0,NaN,NaN,-38.862200,-73.989200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,gSE0qlewC1j4UoKOU2+A5amKtzc1j/I5h2bhjv2d+Q8=
4,NaN,"{""textFieldMeasurementError"": 0.005, ""ChoiceDi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,P.N. Tolhuaca,P.N. Tolhuaca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.215711,-71.806722,(WGS84) World Geodetic System 1984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,Tw11N7qFI/XzbJam2Z1HHIoLjzOfoRsDpZEPw5+W4tU=
5,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,420.0,433.0,NaN,NaN,-38.755300,-74.015000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,En2a0PyVD7aactcJD76RHyDU8sr/h1A0JJMRO3G3aZs=
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,La Fusta,La Fusta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SjGfyCIgmOvy0/TRjLyF0Pvc7GHh+wJlKN42igGQvg0=
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Niblinto,Niblinto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FcxK8Xhw1Ou0WSDHlOhtlKBTrdsnxeqxolqtNEjue0E=
8,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"PN Tolhuaca, sendero",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.230172,-71.726116,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,ptH/BjpA8L3Ks1Xb3MZQT2eIuLgx0KW2Y9hwASH1+PY=
9,NaN,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Lago Budi - Romopulli Huapi,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Romopulli Huapi,-38.840324,-73.302494,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,2bt8LTmCaBOsLwJpKal7zLTWNNp0URLWyq13fBgx4ow=


In [4]:
%%time
#Testing...
DBPrep(data).verbatim_coordinates_autocomplete()
DBPrep(data).verbatim_coordinate_system_autocomplete()
DBPrep(data).verbatim_latitude_longitude_autocomplete()
#DBPrep(data).coordinate_precision_autocomplete() Deprecated function implemented on GeorefCalculator

Wall time: 33 ms


In [5]:
data_pep=data.query("decimalLatitude.notna() and decimalLongitude.notna() ")
for index in data_pep.index:
    verbatimLatitude=data_pep.loc[index,"decimalLatitude"]
    verbatimLongitude=data_pep.loc[index,"decimalLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem="decimal degrees"
    verbatimSRS=data_pep.loc[index,"geodeticDatum"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]
    

In [6]:
data_pep=data.query("verbatimLatitude.notna() and verbatimLongitude.notna() ")
for index in data_pep.index:
    verbatimLatitude=data_pep.loc[index,"verbatimLatitude"]
    verbatimLongitude=data_pep.loc[index,"verbatimLongitude"]
    dynamicProperties=data_pep.loc[index,"dynamicProperties"]
    verbatimCoordinteSystem=data_pep.loc[index,"verbatimCoordinateSystem"]
    verbatimSRS=data_pep.loc[index,"verbatimSRS"]
    SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).georeferencer()
    result_dict=SeleniumGeoref(verbatimCoordinteSystem,verbatimLatitude,verbatimLongitude,dynamicProperties,verbatimSRS).get_results()
    for keys in result_dict.keys():
        data.loc[index,keys]=result_dict[keys]

In [7]:
%%time
# Testing ...
#coord_transform(data).utm_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).dms_to_dg() Deprecated function implemented on GeorefCalculator
#CoordTransform(data).geodetic_datum_autocomplete()  Deprecated function implemented on GeorefCalculator
CoordTransform(data).location_autocomplete() 
CoordTransform(data).continent_autocomplete() 
CoordTransform(data).higher_geography_autocomplete() 

Wall time: 1min 52s


In [8]:
driver.quit()

In [9]:
data

,higherGeography,dynamicProperties,waterBody,continent,country,countryCode,stateProvince,county,municipality,islandGroup,island,locality,verbatimLocality,minimumElevationInMeters,maximumElevationInMeters,verbatimElevation,minimumDepthInMeters,maximumDepthInMeters,verbatimDepth,locationRemarks,decimalLatitude,decimalLongitude,geodeticDatum,coordinateUncertaintyInMeters,coordinatePrecision,footprintSRS,footprintWKT,verbatimCoordinates,verbatimLatitude,verbatimLongitude,verbatimCoordinateSystem,verbatimSRS,georeferencedBy,georeferencedDate,georeferenceProtocol,georeferenceSources,georeferenceVerificationStatus,georeferenceRemarks,localityid,georeferenceProtocol
0,South America | Chile | CL | Araucanía | Malle...,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",NaN,South America,Chile,CL,Araucanía,Malleco,Lonquimay,NaN,NaN,"Lagunita en Ruta 181, 6 km al Este de Lonquimay",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.453318,-71.292183,datum not recorded,2878,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:29.136Z,NaN,Google Earth,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TxmVXS0HuObUcicDzF9JBaWNHK4C/tiPdf9jxpTLMqU=,Georeferencing Quick Reference Guide (Zermogli...
1,South America | Chile | CL | Ñuble | Punilla |...,"{""textFieldMeasurementError"": 5, ""ChoiceDistUn...",NaN,South America,Chile,CL,Ñuble,Punilla,Coihueco,NaN,NaN,"Reserva Nacional Los Huemules de Niblinto, Coi...",R.F. Niblinto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-36.7286777,-71.4969727,(WGS84) World Geodetic System 1984,8321,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:30.449Z,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,WRYd0YGi5ik2bvvJRhC1Ynf4OyfzirN66qbhuD3Fgfc=,Georeferencing Quick Reference Guide (Zermogli...
2,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,150.0,156.0,NaN,NaN,-38.8464,-73.7889,datum not recorded,2695,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:31.559Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,TtMBF2hQMfzrgMVqyIbTHRP0UF9v4Qn8uNonMtuye/s=,Georeferencing Quick Reference Guide (Zermogli...
3,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,464.0,500.0,NaN,NaN,-38.8622,-73.9892,datum not recorded,2695,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:32.828Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,gSE0qlewC1j4UoKOU2+A5amKtzc1j/I5h2bhjv2d+Q8=,Georeferencing Quick Reference Guide (Zermogli...
4,South America | Chile | CL | Araucanía | Malle...,"{""textFieldMeasurementError"": 0.005, ""ChoiceDi...",NaN,South America,Chile,CL,Araucanía,Malleco,Victoria,NaN,NaN,P.N. Tolhuaca,P.N. Tolhuaca,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-38.2157108,-71.8067221,(WGS84) World Geodetic System 1984,13807,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:34.565Z,NaN,NaN,NaN,Georreferenciado a través de mapas de la Bibli...,Tw11N7qFI/XzbJam2Z1HHIoLjzOfoRsDpZEPw5+W4tU=,Georeferencing Quick Reference Guide (Zermogli...
5,South Pacific,"{""textFieldMeasurementError"": 100,""ChoiceDistU...",South Pacific,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Carahue,NaN,0.0,0.0,NaN,420.0,433.0,NaN,NaN,-38.7553,-74.015,datum not recorded,2695,0.0000001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2021-03-20T21:00:35.759Z,NaN,NaN,NaN,Se utilizan coordenadas decimalLatitud/decimal...,En2a0PyVD7aactcJD76RHyDU8sr/h1A0JJMRO3G3aZs=,Georeferencing Quick Reference Guide (Zermogli...
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,La Fusta,La Fusta,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,SjGfyCIgmOvy0/TRjLyF0Pvc7GHh+wJlKN42igGQvg0=,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Niblinto,Niblinto,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FcxK8Xhw1Ou0WSDHlOhtlKBTrdsnxeqxolqtNEjue0E=,NaN
8,South America | Chile | CL | Arau